In [3]:
import requests
from time import sleep
import numpy as np
import time
import statistics

s = requests.Session()
s.headers.update({'X-API-key': '9EN88DGS'}) # Make sure you use YOUR API Key

# global variables
MAX_LONG_EXPOSURE_NET = 25000
MAX_SHORT_EXPOSURE_NET = -25000
MAX_EXPOSURE_GROSS = 500000
ORDER_LIMIT = 25000/2

def get_tick():   
    resp = s.get('http://localhost:9999/v1/case')
    if resp.ok:
        case = resp.json()
        return case['tick'], case['status']


def get_bid_ask(ticker):
    payload = {'ticker': ticker}
    resp = s.get ('http://localhost:9999/v1/securities/book', params = payload)
    if resp.ok:
        book = resp.json()
        bid_side_book = book['bids']
        ask_side_book = book['asks']
        
        bid_prices_book = [item["price"] for item in bid_side_book]
        ask_prices_book = [item['price'] for item in ask_side_book]
        
        best_bid_price = bid_prices_book[0]
        best_ask_price = ask_prices_book[0]
  
        return best_bid_price, best_ask_price

def get_time_sales(ticker):
    payload = {'ticker': ticker}
    resp = s.get ('http://localhost:9999/v1/securities/tas', params = payload)
    if resp.ok:
        book = resp.json()
        time_sales_book = [item["quantity"] for item in book]
        return time_sales_book

def get_position():
    resp = s.get ('http://localhost:9999/v1/securities')
    if resp.ok:
        book = resp.json()
        gross_position = abs(book[1]['position']) + abs(book[2]['position']) + 2 * abs(book[3]['position'])
        net_position = book[1]['position'] + book[2]['position'] + 2 * book[3]['position']
        return gross_position, net_position


# New function
def get_vol_asset(asset_index):
    resp = s.get ('http://localhost:9999/v1/securities')
    if resp.ok:
        book = resp.json()
        return book[asset_index]['position']


def get_open_orders(ticker):
    payload = {'ticker': ticker}
    resp = s.get ('http://localhost:9999/v1/orders', params = payload)
    if resp.ok:
        orders = resp.json()
        buy_orders = [item for item in orders if item["action"] == "BUY"]
        sell_orders = [item for item in orders if item["action"] == "SELL"]
        return buy_orders, sell_orders

def get_order_status(order_id):
    resp = s.get ('http://localhost:9999/v1/orders' + '/' + str(order_id))
    if resp.ok:
        order = resp.json()
        return order['status']
    

# New function
def create_etf(vol_rgld, vol_rfin, id_creation):
    resp = s.post(f'http://localhost:9999/v1/leases/{id_creation}', params={'from1': 'RGLD', 'quantity1': vol_rgld, 'from2': 'RFIN', 'quantity2': vol_rfin})
    #if resp.ok:
        #print(f"ETF Created: {resp.json()}")

# New function
def redeem_etf(vol_indx, vol_cad, id_redemption):
    resp = s.post(f'http://localhost:9999/v1/leases/{id_redemption}', params={'from1': 'INDX', 'quantity1': vol_indx, 'from2': 'CAD', 'quantity2': vol_cad})
    #if resp.ok:
        #print(f"ETF Redeemed: {resp.json()}")


def get_lease():
    resp = s.post('http://localhost:9999/v1/leases', params = {'ticker': 'ETF-Creation'})
    resp = s.post('http://localhost:9999/v1/leases', params = {'ticker': 'ETF-Redemption'})
    resp = s.get('http://localhost:9999/v1/leases')
    leases = resp.json()
    id_creation = 0
    id_redemption = 0

    for lease in leases:
        if lease['ticker'] == 'ETF-Creation':
            id_creation = lease['id']
        if lease['ticker'] == 'ETF-Redemption':
            id_redemption = lease['id']
    return id_creation, id_redemption

def main():
    tick, status = get_tick()
    ticker_list = ['RGLD','RFIN','INDX']
    market_prices = np.array([0.,0.,0.,0.,0.,0.])
    market_prices = market_prices.reshape(3,2)
    spreads = []

    id_creation = 0
    id_redemption = 0
    while id_creation == 0 or id_redemption == 0:
        print(id_creation, id_redemption)
        id_creation, id_redemption = get_lease()

    while status == 'ACTIVE':        
        starttime = time.time()
        for i in range(3):
            
            ticker_symbol = ticker_list[i]
            market_prices[i,0], market_prices[i,1] = get_bid_ask(ticker_symbol)        

        

        # Spread for selling or buying the stocks (vice versa for ETF)
        sell_spread = market_prices[0, 0] + market_prices[1, 0] - market_prices[2, 1]
        buy_spread = (market_prices[2, 0] + 0.0375) - (market_prices[0, 1] + market_prices[1, 1])

        if sell_spread >= 0.05 and sell_spread < 0.08:
            resp = s.post('http://localhost:9999/v1/orders', params = {'ticker': 'RGLD', 'type': 'MARKET', 'quantity': ORDER_LIMIT/4, 'price': market_prices[0, 1], 'action': 'SELL'})
            resp = s.post('http://localhost:9999/v1/orders', params = {'ticker': 'RFIN', 'type': 'MARKET', 'quantity': ORDER_LIMIT/4, 'price': market_prices[1, 1], 'action': 'SELL'})
            resp = s.post('http://localhost:9999/v1/orders', params = {'ticker': 'INDX', 'type': 'MARKET', 'quantity': ORDER_LIMIT/4, 'price': market_prices[2, 0], 'action': 'BUY'})
            
        if buy_spread >= 0.05 and buy_spread < 0.08:
            resp = s.post('http://localhost:9999/v1/orders', params = {'ticker': 'RGLD', 'type': 'MARKET', 'quantity': ORDER_LIMIT/4, 'price': market_prices[0, 0], 'action': 'BUY'})
            resp = s.post('http://localhost:9999/v1/orders', params = {'ticker': 'RFIN', 'type': 'MARKET', 'quantity': ORDER_LIMIT/4, 'price': market_prices[1, 0], 'action': 'BUY'})
            resp = s.post('http://localhost:9999/v1/orders', params = {'ticker': 'INDX', 'type': 'MARKET', 'quantity': ORDER_LIMIT/4, 'price': market_prices[2, 1], 'action': 'SELL'})

        if sell_spread >= 0.08 and sell_spread < 0.10:
            resp = s.post('http://localhost:9999/v1/orders', params = {'ticker': 'RGLD', 'type': 'MARKET', 'quantity': ORDER_LIMIT/2, 'price': market_prices[0, 1], 'action': 'SELL'})
            resp = s.post('http://localhost:9999/v1/orders', params = {'ticker': 'RFIN', 'type': 'MARKET', 'quantity': ORDER_LIMIT/2, 'price': market_prices[1, 1], 'action': 'SELL'})
            resp = s.post('http://localhost:9999/v1/orders', params = {'ticker': 'INDX', 'type': 'MARKET', 'quantity': ORDER_LIMIT/2, 'price': market_prices[2, 0], 'action': 'BUY'})
            
        if buy_spread >= 0.08 and buy_spread < 0.10:
            resp = s.post('http://localhost:9999/v1/orders', params = {'ticker': 'RGLD', 'type': 'MARKET', 'quantity': ORDER_LIMIT/2, 'price': market_prices[0, 0], 'action': 'BUY'})
            resp = s.post('http://localhost:9999/v1/orders', params = {'ticker': 'RFIN', 'type': 'MARKET', 'quantity': ORDER_LIMIT/2, 'price': market_prices[1, 0], 'action': 'BUY'})
            resp = s.post('http://localhost:9999/v1/orders', params = {'ticker': 'INDX', 'type': 'MARKET', 'quantity': ORDER_LIMIT/2, 'price': market_prices[2, 1], 'action': 'SELL'})

        if sell_spread >=0.10:
            resp = s.post('http://localhost:9999/v1/orders', params = {'ticker': 'RGLD', 'type': 'MARKET', 'quantity': ORDER_LIMIT, 'price': market_prices[0, 1], 'action': 'SELL'})
            resp = s.post('http://localhost:9999/v1/orders', params = {'ticker': 'RFIN', 'type': 'MARKET', 'quantity': ORDER_LIMIT, 'price': market_prices[1, 1], 'action': 'SELL'})
            resp = s.post('http://localhost:9999/v1/orders', params = {'ticker': 'INDX', 'type': 'MARKET', 'quantity': ORDER_LIMIT, 'price': market_prices[2, 0], 'action': 'BUY'})
            
        if buy_spread >=0.08:
            resp = s.post('http://localhost:9999/v1/orders', params = {'ticker': 'RGLD', 'type': 'MARKET', 'quantity': ORDER_LIMIT, 'price': market_prices[0, 0], 'action': 'BUY'})
            resp = s.post('http://localhost:9999/v1/orders', params = {'ticker': 'RFIN', 'type': 'MARKET', 'quantity': ORDER_LIMIT, 'price': market_prices[1, 0], 'action': 'BUY'})
            resp = s.post('http://localhost:9999/v1/orders', params = {'ticker': 'INDX', 'type': 'MARKET', 'quantity': ORDER_LIMIT, 'price': market_prices[2, 1], 'action': 'SELL'})
        
        spreads.append(abs(market_prices[0, 1] + market_prices[1, 1] - market_prices[2, 0]))
        barrier = get_vol_asset(3) # how much INDX we are holding

        #if abs(barrier) > 20000:
        if barrier > 0:
            # If long INDX, redeem INDX to make stocks
            vol_indx = barrier
            vol_cad = vol_indx * 0.0375
            redeem_etf(vol_indx, vol_cad, id_redemption)

        if barrier < 0:
            # If shorting of INDX, create INDX with stocks
            vol_rgld = -1 * barrier
            vol_rfin = -1 * barrier
            create_etf(vol_rgld, vol_rfin, id_creation)  
        # Reset start time
        #print(starttime + 2 - time.time())W

        
        tick, status = get_tick()
        difference = time.time() - starttime
        #print(float(difference))
        sleep(0.5 - difference) 
        print(statistics.mean(spreads))

if __name__ == '__main__':
    main()

0 0
0 0
0.0899999999999963
0.0899999999999963
0.06666666666666406
0.05249999999999755
0.05599999999999809
0.058333333333331794
0.05999999999999872
0.058749999999998526
0.062222222222220895
0.061999999999999035
0.06181818181818115
0.060833333333332774
0.06076923076923043
0.06571428571428559
0.07066666666666659
0.07374999999999998
0.0770588235294117
0.07999999999999988
0.08368421052631578
0.0870000000000001
0.08857142857142872
0.09000000000000018
0.09130434782608718
0.09333333333333356
0.09560000000000031
0.09769230769230806
0.09962962962963007
0.10107142857142902
0.10241379310344872
0.10366666666666711
0.10483870967741982
0.10562500000000041
0.10666666666666698
0.10705882352941198
0.10714285714285735
0.10861111111111141
0.11000000000000039
0.11131578947368469
0.11256410256410312
0.11375000000000055
0.11487804878048835
0.11595238095238149
0.11744186046511679
0.11886363636363685
0.11933333333333383
0.12000000000000045
0.11978723404255362
0.11958333333333375
0.11938775510204121
0.119200000

IndexError: list index out of range